# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [74]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vorgashor,67.5833,63.9500,58.28,66,96,7.11,RU,1724786423
1,1,blackmans bay,-43.0167,147.3167,46.53,88,100,1.99,AU,1724786424
2,2,la passe,45.5549,-0.8967,70.74,59,67,10.49,FR,1724786425
3,3,mogocha,53.7333,119.7667,57.79,97,100,3.44,RU,1724786426
4,4,nova vicosa,-17.8919,-39.3719,72.86,76,100,15.52,BR,1724786429
...,...,...,...,...,...,...,...,...,...,...
571,571,la plata,-34.9215,-57.9545,58.35,55,0,5.99,AR,1724787094
572,572,forbes,-33.3833,148.0167,53.35,45,100,12.59,AU,1724787095
573,573,constitucion,-35.3333,-72.4167,57.63,60,100,13.35,CL,1724787080
574,574,port lincoln,-34.7333,135.8667,55.36,62,0,22.12,AU,1724787098


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [75]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(x='Lng', y='Lat', color='City', geo=True,tiles="OSM", frame_width=700,frame_height=500, size='Humidity', scale=0.9, hover_cols=['City','Country','Humidity'])
                                     

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [89]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp = (city_data_df['Max Temp'] >= 21) & (city_data_df['Max Temp'] <= 27)
cloudiness = city_data_df['Cloudiness'] = 0
wind_speed = city_data_df['Wind Speed'] < 4.5
filtered_df = city_data_df[max_temp & cloudiness & wind_speed]
weather_df = filtered_df.copy()

# Drop any rows with null values
weather_df.dropna(inplace=True)

# Display sample data
weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 3: Create a new DataFrame called `hotel_df`.

In [90]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [91]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 10
params = {
    "categories": categories,
    "radius": radius,
    "limit": limit, 
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = row['Lng']
    lat = row['Lat']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}" 
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


,City,Country,Lat,Lng,Humidity,Hotel Name


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [92]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(x='Lng',y='Lat', color='City', geo=True,tiles="OSM",  frame_width=1000,frame_height=750,size='Humidity',scale= 2,hover_cols=['City','Country','Hotel Name'])

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)